In [1]:
# import numpy 
import numpy as np 
# import pandas
import pandas as pd 
# import regex
import re
# import nltk
import nltk 

In [2]:
# load data
data_source_url = "https://raw.githubusercontent.com/kolaveridi/kaggle-Twitter-US-Airline-Sentiment-/master/Tweets.csv"
airline_tweets = pd.read_csv(data_source_url)

* print top 5 rows

In [3]:
airline_tweets.head()


,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


* use 'text' column to create array with name 'features'
* use 'airline_sentiment' column to create array with name 'labels'

In [4]:
features = airline_tweets.text
labels = airline_tweets.airline_sentiment

In [5]:
labels

0         neutral
1        positive
2         neutral
3        negative
4        negative
           ...   
14635    positive
14636    negative
14637     neutral
14638    negative
14639     neutral
Name: airline_sentiment, Length: 14640, dtype: object

* clean the text in features array
* Remove all the special characters
* remove all single characters
* Remove single characters from the start
* Substituting multiple spaces with single space
* Removing prefixed 'b'
* Converting to Lowercase

In [6]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [7]:
processed_features = []

for sentence in features:

    words = sentence.lower().split()
    # Remove all the special characters
    table = str.maketrans('', '', string.punctuation)
    words = [word.translate(table) for word in words]

    # remove all single characters
    words = [word for word in words if len(word) > 1]
    # creating the sentence again
    cleanedSentence = ''
    for word in words:
        cleanedSentence += word+ ' '
    # Removing prefixed 'b'
    processed_features.append(cleanedSentence) 
    

In [8]:
airline_tweets['processed_features'] = processed_features

In [9]:
airline_tweets['processed_features'].shape

(14640,)

* import stopwords from nltk

In [10]:
from nltk.corpus import stopwords

* import TfidfVectorizer from sklearn

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

* create vectorizer inastance of TfidfVectorizer with following settings:
* max_features=2500, min_df=7, max_df=0.8, stop_words=stopwords.words('english')

In [12]:
vector = TfidfVectorizer(max_features=2500, min_df=7, max_df=0.8, stop_words=stopwords.words('english'))

* transform features with vectorizer

In [17]:
cleanedFeatures = vector.fit_transform(airline_tweets.processed_features)

* import train_test_split from sklearn and split the data

In [18]:
 X_train, X_test, y_train, y_test = train_test_split(cleanedFeatures, labels, random_state=42)

* import classifier of your choice from sklearn (e.g. Random Forest, LogReg, Naive Bayes)

In [22]:
from sklearn import naive_bayes
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
lR = LogisticRegression().fit(X_train,y_train)

rf = RandomForestClassifier().fit(X_train,y_train)

clf = naive_bayes.MultinomialNB().fit(X_train,y_train)

* fit your classifier to data

* predict X_test

In [26]:
from sklearn.metrics import roc_auc_score
print( 'Bayes' ,roc_auc_score(y_test,clf.predict_proba(X_test),multi_class='ovo'))
print( 'rf' ,roc_auc_score(y_test,rf.predict_proba(X_test),multi_class='ovo'))
print( 'lr' ,roc_auc_score(y_test,lR.predict_proba(X_test),multi_class='ovo'))

Bayes 0.8649725813424024
rf 0.853174663582774
lr 0.8886028931551101


* import confusion matrix and accuracy_score

In [27]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

* print confusion matrix

In [42]:
clf.predict_proba(X_test)[:,1]

array([0.07194807, 0.11170358, 0.11638509, ..., 0.31292958, 0.0403359 ,
       0.39847164])

* print accaccuracy_score